### Getting frontal images of studies which have two or more frontal-lateral images.
Comparing train.csv to pair-wise.csv

In [1]:
import pandas as pd
import numpy as np

#1= positive; 0: negetive; -1: uncertain
trainDF = pd.read_csv('CheXpert-v1.0-small/train.csv', dtype=str)
trainDF = trainDF.fillna(value = '0.0')
# trainDF = trainDF.replace('0.0', '0')
# trainDF = trainDF.replace('-1.0', '-1')
# trainDF = trainDF.replace('1.0', '1')
pairwiseDF = pd.read_csv('pair-wise.csv', dtype=str)
pairwiseDF

,Path,No_Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural_Effusion
0,pair-wise/patient00002-study1,0.0,0.0,0.0,-1.0,0.0,0.0
1,pair-wise/patient00004-study1,1.0,0.0,0.0,0.0,0.0,0.0
2,pair-wise/patient00005-study1,1.0,0.0,0.0,0.0,0.0,0.0
3,pair-wise/patient00009-study1,0.0,1.0,1.0,0.0,0.0,0.0
4,pair-wise/patient00010-study1,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
33146,pair-wise/patient60704-study1-L-0,0.0,0.0,-1.0,0.0,0.0,0.0
33147,pair-wise/patient60704-study1-L-2,0.0,0.0,-1.0,0.0,0.0,0.0
33148,pair-wise/patient60706-study1,0.0,0.0,1.0,0.0,0.0,0.0
33149,pair-wise/patient60707-study1,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
pairwiseDF['Path'] = pairwiseDF['Path'].str.rsplit('/', 1, expand = True)[1]

In [11]:
x = pairwiseDF['Path'].str.rsplit('-', 3, expand = True)
pairwiseDF['Path'] = x[[0, 1]].agg('/'.join, axis=1)
pairwiseDF.drop_duplicates(subset=['Path'])
pairwiseDF

,Path,No_Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural_Effusion
0,patient00002/study1,0.0,0.0,0.0,-1.0,0.0,0.0
1,patient00004/study1,1.0,0.0,0.0,0.0,0.0,0.0
2,patient00005/study1,1.0,0.0,0.0,0.0,0.0,0.0
3,patient00009/study1,0.0,1.0,1.0,0.0,0.0,0.0
4,patient00010/study1,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
33146,patient60704/study1,0.0,0.0,-1.0,0.0,0.0,0.0
33147,patient60704/study1,0.0,0.0,-1.0,0.0,0.0,0.0
33148,patient60706/study1,0.0,0.0,1.0,0.0,0.0,0.0
33149,patient60707/study1,0.0,0.0,1.0,0.0,0.0,0.0


In [2]:
#removing lateral image entries
indexLateral = trainDF[trainDF['Frontal/Lateral'] == 'Lateral' ].index
trainDF = trainDF.drop(indexLateral)
trainDF.head(5)

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,0.0,0.0,-1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,1.0,0.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
from tqdm.notebook import trange, tqdm
tqdm.pandas()

count = 0
newFrontalOnlyDF = trainDF.copy()
pairwiseDF.Path.astype('str')
for idx, row in tqdm(trainDF.iterrows(), total=df.shape[0]):
    tmp = str((row['Path'].split('/'))[2]+'/'+(row['Path'].split('/'))[3])
    #tmp = "1.0"
    if pairwiseDF['Path'].str.contains(tmp).any() == False:
        count += 1
        newFrontalOnlyDF = newFrontalOnlyDF.drop(index = idx)
print(count)

/Users/zubin/anaconda3/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel



157797


In [73]:
newFrontalOnlyDF.describe()

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
count,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230,33230
unique,33230,2,73,1,3,2,3,3,3,3,3,3,3,3,3,3,3,3,3
top,CheXpert-v1.0-small/train/patient13471/study1/...,Male,90,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
freq,1,21067,1001,33230,27139,27753,29406,28087,20281,30660,29200,27921,29025,25434,30889,21491,31326,31470,23644


In [74]:
pairwiseDF.describe()

,Path,No_Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural_Effusion
count,33151,33151,33151,33151,33151,33151,33151
unique,31413,2,3,3,3,3,3
top,patient13559/study1,0.0,0.0,0.0,0.0,0.0,0.0
freq,2,27625,25456,28049,27870,29341,21520


In [75]:
newFrontalOnlyDF.to_csv("train-FrontalOnly-OfPairs-CXP.csv", index=False)